In [1]:
!pip install simpy


In [1]:
import simpy
import random
import pandas as pd
import random

def simulate_hiring_process(hr_capacity=1, interviewer_capacity=1, num_resumes=30):
    results = []

    def hiring_process(env, name, hr, interviewer):
        arrival_time = env.now
        with hr.request() as req:
            yield req
            yield env.timeout(random.expovariate(1.0 / 20))  # 篩選

        with interviewer.request() as req2:
            yield req2
            yield env.timeout(random.expovariate(1.0 / 30))  # 面試

        yield env.timeout(random.expovariate(1.0 / 10))  # 發 offer
        total_time = env.now - arrival_time
        results.append({"resume": name, "total_process_time": total_time})

    def resume_generator(env, hr, interviewer):
        for i in range(num_resumes):
            env.process(hiring_process(env, f"R{i+1}", hr, interviewer))
            yield env.timeout(random.expovariate(1.0 / 10))  # 到件間隔

    env = simpy.Environment()
    hr = simpy.Resource(env, capacity=hr_capacity)
    interviewer = simpy.Resource(env, capacity=interviewer_capacity)

    env.process(resume_generator(env, hr, interviewer))
    env.run()

    return pd.DataFrame(results)


In [2]:
random.seed(42)
df_1_1 = simulate_hiring_process(1, 1, num_resumes=10)
df_1_1["scenario"] = "HR:1, Interviewer:1"
df_1_1.describe()


,total_process_time
count,10.000000
mean,76.070417
std,65.826669
min,12.681161
25%,33.178371
50%,52.521538
75%,88.170039
max,222.891012


In [3]:
random.seed(42)
df_2_1 = simulate_hiring_process(2, 1, num_resumes=10)
df_2_1["scenario"] = "HR:2, Interviewer:1"

random.seed(42)
df_1_2 = simulate_hiring_process(1, 2, num_resumes=10)
df_1_2["scenario"] = "HR:1, Interviewer:2"

print("HR:2, Interviewer:1")
print(df_2_1.describe())

print("\nHR:1, Interviewer:2")
print(df_1_2.describe())


HR:2, Interviewer:1
       total_process_time
count           10.000000
mean            71.506997
std             58.201148
min             12.681161
25%             37.514142
50%             43.273007
75%            107.175774
max            199.793081

HR:1, Interviewer:2
       total_process_time
count           10.000000
mean            62.300827
std             38.550166
min             12.681161
25%             28.737120
50%             61.071478
75%             93.038033
max            114.180883
